# Лабораторна робота 1

**Тема:** Прийняття рішення в умовах ризику

**Мета роботи:** дослідження питань ухвалення рішення в умовах, коли вибір деякої стратегії гарантує одержання бажаного результату з певною ймовірністю.

**Виконав:** студент 321 групи, Ткачук І.В.

## Завдання 

### Варіант 6

У сільському господарстві з посівною площею 1430 га вирішено
побудувати елеватор за одним з типових проектів на 20, 30, 40, 50 або 60
тис. центнерів зерна. Будівництво елеватора обійдеться в 370000 грн.
Вартість матеріалів і устаткування для елеватора потужністю 20 тис.
дорівнює 600000 грн. і зростає на 10% із зростанням об'єму на 10 тис.
Витрати на експлуатацію елеватора на 20 тис. дорівнюють 200 000 грн. і
зростають на 10000 грн. із зростанням об'єму на 10 тис. За зберігання зерна
на рахунок елеватора вноситься плата 100 грн за центнер. Зібрано
статистику врожайності за останні 10 років. Який із проектів вибрати?

| Рік | 2008 | 2009 | 2010 | 2011 | 2012 | 2013 | 2014 | 2015 | 2016 | 2017 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Урожайність, ц/га | 14.4 | 20.1 | 15.5 | 34.5 | 55.4 | 26.2 | 12.2 | 76.0 | 22.5| 33.5|

In [7]:
import pandas as pd
from IPython.display import display, HTML

# Дані врожайності
yield_data = {
    "Year": [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017],
    "Yield": [14.4, 20.1, 15.5, 34.5, 55.4, 26.2, 12.2, 76.0, 25.5, 33.5]
}

yield_df = pd.DataFrame(yield_data)

# Загальна посівна площа у гектарах
sown_area = 1430

elavator_capacities = [20_000, 30_000, 40_000, 50_000, 60_000]

initial_costs = {20_000: 600_000}

operating_costs = {20_000: 200_000}

cost_increase_rate = 0.1  # 10%

operating_cost_increase = 10000 

storage_fee_per_centner = 100

yield_df["Total yield"] = yield_df["Yield"] * sown_area

yield_df 



,Year,Yield,Total yield
0,2008,14.4,20592.0
1,2009,20.1,28743.0
2,2010,15.5,22165.0
3,2011,34.5,49335.0
4,2012,55.4,79222.0
5,2013,26.2,37466.0
6,2014,12.2,17446.0
7,2015,76.0,108680.0
8,2016,25.5,36465.0
9,2017,33.5,47905.0


Знайдемо вартість матеріалів і устаткування для кожного елеватора

In [8]:
for capacity in elavator_capacities[1:]:
    previous_capacity = capacity - 10_000
    initial_costs[capacity] = round(initial_costs[previous_capacity] * (1 + cost_increase_rate))

initial_costs

{20000: 600000, 30000: 660000, 40000: 726000, 50000: 798600, 60000: 878460}

Тепер визначимо витрати на експлутацію

In [9]:
for capacity in elavator_capacities[1:]:
    operating_costs[capacity] = operating_costs[capacity - 10_000] + operating_cost_increase

operating_costs 

{20000: 200000, 30000: 210000, 40000: 220000, 50000: 230000, 60000: 240000}

Розраховуємо прибуток, збитки для кожної стратегії

In [10]:
results = []

for capacity in elavator_capacities:
    total_cost = initial_costs[capacity] + operating_costs[capacity] - 370_000

    for index, row in yield_df.iterrows():
        if row["Total yield"] <= capacity:
            profit = row["Total yield"] * storage_fee_per_centner - total_cost
        else:
            profit = (row["Total yield"] * storage_fee_per_centner
                    - 370_000
                    + 600_000 * 1.1 ** (capacity / 10 - 2)
                    + 200_000
                    + 10_000 * (capacity / 10 - 2))

        results.append({"Year": row["Year"], "Capacity": capacity, "Profit": profit})

results_df = pd.DataFrame(results)

grouped_data = results_df.groupby("Capacity")

html_str = ""

for capacity, data in grouped_data:
    html_table = (
        data.drop(["Capacity"], axis=1).reset_index(drop=True).to_html()
    )
    html_str += f"""
    <div style='display: inline-block; padding: 10px;'>
        <h2>Елеватор {capacity}</h2>
        {html_table}
    </div>"""

display(HTML(html_str))

,Year,Profit
0,2008.0,3.025075e+88
1,2009.0,3.025075e+88
2,2010.0,3.025075e+88
3,2011.0,3.025075e+88
4,2012.0,3.025075e+88
5,2013.0,3.025075e+88
6,2014.0,1.314600e+06
7,2015.0,3.025075e+88
8,2016.0,3.025075e+88
9,2017.0,3.025075e+88


In [11]:
avg_profit_loss_by_capacity = results_df.groupby('Capacity')['Profit'].mean()
avg_profit_loss_by_capacity

Capacity
20000     2.722568e+88
30000    4.483040e+129
40000    7.381872e+170
50000    9.116364e+211
60000    2.251681e+253
Name: Profit, dtype: float64

In [12]:
best_strategy = avg_profit_loss_by_capacity.idxmax()
best_strategy

60000

## Контрольні запитання

1. Сформулюйте формально модель задачі прийняття рішення в умовах ризику.
2. Що називається функцією корисності?
3. Яким аксіомам задовольняє функція корисності U(xi )?
4. Як обчислити умовні ймовірності?
5. Як обчислити очікувану корисність для кожної стратегії?

### 1. **Формальна модель задачі прийняття рішення в умовах ризику:**

- **Елементи моделі:** включають набір альтернативних стратегій або дій, набір можливих станів світу (або результатів), які можуть настати, та відповідні їм ймовірності.
- **Рішення:** Рішення полягає в виборі стратегії, яка максимізує очікувану корисність або інший критерій оптимальності, враховуючи ризики та ймовірності різних станів світу.

### 2. **Функція корисності:**

Це математична функція, яка відображає величину задоволення або вигоди, яку індивід отримує від різних результатів або наборів результатів. Функція корисності дозволяє кількісно оцінити переваги та визначити кращий вибір з точки зору індивіда.

### 3. **Аксіоми функції корисності $U(x_i$)**:

- **Повнота:** Кожна пара результатів може бути порівнняна.
- **Транзитивність:** Якщо результат $A$ кращий за $B$, і $B$ кращий за $C$, тоді $A$ кращий за $C$
- **Неперервність:** Невеликі зміни у результаті ведуть до невеликих змін у корисності
- **Незалежність:** Переваги між результатми є незалежними від інших факторів

### 4. **Обчислення умовних ймовірностей:**

Умовнай ймовірність $P(A|B)$ обчислюється як $\frac{P(A \cap B)}{P(B)}$, де $P(A \cap B)$ - це ймовірність того, що відбудеться і $A$ і $B$, а ймовірність $P(B)$ - ймовірність настання події $B$.

### 5. **Обчислення очікуваної корисності для кожної стратегії:**

Очікувана корисність для стратегії обчислюється як сума добутків ймовірностей кожного можливого стану світу на корисність, пов'язану з цим станом при даній стратегії. Формула: $EU = \sum P(x_i) \cdot U(x_i)$, де $EU$ – очікувана корисність, $P(x_i)$ – ймовірність стану $x_i$, і $U(x_i)$ – корисність від стану $x_i$.
